<a href="https://colab.research.google.com/github/JERSONMALDONA/senales-y-sistemas/blob/main/taler_2/Dashboard_Detectordeaudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Detector de genero musical en Streamlit.

Utilizando la herramienta Streamlit, genere un dashboard
para los ejercicios: i) Aplicacion en comunicaciones - modulacion AM  y ii) Aplicacion en circuitos electricos - potencia (Ver material de apoyo Dashboards).
Link detector de Youtube: https://github.com/jmoralespineda/SENALES_Y_SISTEMAS/blob/main/youtube_detector.ipynb texto en negrita

In [1]:
!pip install streamlit librosa joblib pyngrok yt-dlp gdown
!apt-get install -y ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
%%writefile app.py
import streamlit as st
import librosa
import librosa.display
import numpy as np
import joblib
import os
import yt_dlp
import gdown  # para descargar desde google drive
import matplotlib.pyplot as plt

# https://drive.google.com/file/d/1_NP1A7JbMB4tkIU7JCfRUTNurc6fd7xQ/view?usp=sharing

st.set_page_config(page_title="Detector de Género Musical", page_icon="🎵")
st.title(" Detector de Género Musical: reggaeton_vs_metal")

# 🔗 id del archivo en google drive
FILE_ID = "1_NP1A7JbMB4tkIU7JCfRUTNurc6fd7xQ"
modelo_path = "/content/reggaeton_vs_metal.pkl"

# leer el modelo desde el archivo
if not os.path.exists(modelo_path):
    try:
        st.info("📥 Descargando modelo desde Google Drive...")
        gdown.download(f"https://drive.google.com/uc?id={FILE_ID}", modelo_path, quiet=False)
    except Exception as e:
        st.error(f"❌ Error descargando el modelo: {e}")
        st.stop()

# leer el modelo desde el archivo
try:
    modelo_dict = joblib.load(modelo_path)
    modelo = modelo_dict['modelo'] if isinstance(modelo_dict, dict) else modelo_dict
    clases = modelo_dict['type'] if isinstance(modelo_dict, dict) and 'type' in modelo_dict else None
    st.success("✅ Modelo cargado correctamente.")

except Exception as e:
    st.error(f"❌ Error cargando el modelo: {e}")
    st.stop()

# bajar el sonido desde YouTube y convertirlo a mp3
def download_ytvid_as_mp3(video_url, name):
    output = f"/content/{name}.%(ext)s"
    options = {
        'format': 'bestaudio/best',
        'outtmpl': output,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    st.write(f"🎥 Descargando desde: {video_url}")

    with yt_dlp.YoutubeDL(options) as ydl:
        try:
            ydl.download([video_url])
            final_path = f"/content/{name}.mp3"
            st.write(f"✅ Audio descargado en: {final_path}")
            return final_path if os.path.exists(final_path) else None
        except Exception as e:
            st.error(f"❌ Error al descargar audio: {e}")
            return None

# obtener los datos en frecuencia de forma normalizada
def extraer_fft(file_path):
    try:
        y, sr = librosa.load(file_path, sr=22050)
        fft = np.abs(np.fft.fft(y))
        fft = fft[:len(fft)//2]

        if np.max(fft) != 0:
            fft = fft / np.max(fft)

        if len(fft) < 120001:
            fft = np.pad(fft, (0, 120001 - len(fft)), mode='constant')
        else:
            fft = fft[:120001]

        return fft.reshape(1, -1), y, sr
    except Exception as e:
        st.error(f"❌ Error al procesar el audio: {e}")
        return None, None, None


url = st.text_input("🔗 Ingresa el enlace de la canción (YouTube):")

if url:
    with st.spinner("⬇️ Descargando audio..."):
        mp3_file = download_ytvid_as_mp3(url, "audio")
        if not mp3_file:
            st.error("❌ El archivo MP3 no fue encontrado después de la descarga.")
            st.stop()

    # poner un panel para escuchar el archivo
    st.subheader("🔊 Reproductor de audio")
    audio_bytes = open(mp3_file, 'rb').read()
    st.audio(audio_bytes, format="audio/mp3")

    with st.spinner("🎛️ Extrayendo características..."):
        features, y, sr = extraer_fft(mp3_file)
        if features is None:
            st.stop()

    # mostrar cómo varía la amplitud en el tiempo
    st.subheader("📉 Forma de onda")
    fig_wave, ax_wave = plt.subplots(figsize=(10, 2))
    librosa.display.waveshow(y, sr=sr, ax=ax_wave, color='mediumseagreen')
    ax_wave.set_xlabel("Tiempo (s)")
    ax_wave.set_ylabel("Amplitud")
    st.pyplot(fig_wave)

    # ver cómo se distribuye la energía en frecuencia
    st.subheader("📊 Espectro de Frecuencia (FFT)")
    fft_vals = np.abs(np.fft.fft(y))[:len(y)//2]
    freqs = np.fft.fftfreq(len(y), 1/sr)[:len(y)//2]
    fig_fft, ax_fft = plt.subplots(figsize=(10, 3))
    ax_fft.plot(freqs, fft_vals, color='coral')
    ax_fft.set_xlim(0, 8000)
    ax_fft.set_xlabel("Frecuencia (Hz)")
    ax_fft.set_ylabel("Magnitud")
    st.pyplot(fig_fft)


    clases = ["Reguetón", "Salsa"]
    with st.spinner("🔍 Clasificando género musical..."):
      try:
          pred = modelo.predict(features)
          st.write("Predicción sin índice:", pred)
          pred = pred[0]
          st.write("Predicción valor único:", pred)
          if clases is not None:
             genero = clases[int(pred)-1]
          else:
             genero = "Desconocido"
          st.success(f"🎵 Género detectado: **{genero}**")
      except Exception as e:
          st.error(f"❌ Error al clasificar: {e}")


Writing app.py


In [4]:
from pyngrok import ngrok, conf
import os
import threading

# cerramos conexiones anteriores
ngrok.kill()

# token de acceso ya está listo
ngrok.set_auth_token("2yZeubgB9ZuQOO0rtmZkk2wb7tX_35reVTFMYv8vXhUsoHzxg")

# lanzar la app sin bloquear el flujo
def run():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run)
thread.start()

# crear un enlace web para acceder a la app
public_url = ngrok.connect(8501)
print(f"App publicada: {public_url}")



ERROR:pyngrok.process.ngrok:t=2025-06-16T04:47:42+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-16T04:47:42+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.